# Install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/GitHub

/content/drive/MyDrive/GitHub


In [3]:
repository = "NAS_project"

In [4]:
%cd {repository}

/content/drive/MyDrive/GitHub/NAS_project


In [ ]:
pip install nats_bench

In [ ]:
pip install yacs

In [ ]:
pip install simplejson

In [ ]:
pip install xautodl

In [ ]:
pip install pytorchcv

In [10]:
%env TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch

env: TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch


# import and functions

In [11]:
# import librerie

# librerie già presenti nelle prime versioni
import torch
import argparse
import datasets 
import nasspace
import pandas as pd
import csv
import random
import numpy as np
from timeit import default_timer as timer
import os
from scipy import stats
from nats_bench import create
import xautodl  # import this lib -- "https://github.com/D-X-Y/AutoDL-Projects", you can use pip install xautodl
from xautodl.models import get_cell_based_tiny_net
from sklearn.preprocessing import StandardScaler
from numpy import inf

#librerie nuove
from pycls.models.nas.nas import Cell
import pandas as pd
from pytorchcv.model_provider import get_model as ptcv_get_model
from pytorchcv.model_provider import _models as ptcv_models
from pruners import *
from pruners import predictive
from tqdm import trange
from statistics import mean
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from timeit import default_timer as timer

from GenNAS.builder_task import *
from GenNAS.builder_model import *
from GenNAS.builder_evaluator import *
from GenNAS.utils.config_generator import *


In [13]:
#SELECT DATASET
#Dataset_name = "Cifar-10"
#Dataset_name = "Cifar-100"
Dataset_name = "ImageNet"

In [14]:
GPU = '0'
seed = 1
batch_size = 128

#CIFAR10
if Dataset_name == "Cifar-10":
  dataset = 'cifar10'
  data_loc = 'insert path cifar-10'


#CIFAR100
if Dataset_name == "Cifar-100":
  dataset = 'cifar100'
  data_loc = "insert path cifar-100"

#IMAGENET
if Dataset_name == "ImageNet":
  dataset = 'ImageNet16-120'
  data_loc = "insert path ImageNet16"



In [15]:
os.environ['CUDA_VISIBLE_DEVICES'] = GPU

# Reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
import torch


def get_naswot(network, x):
  network = network.to(device)

  def boolRelu_forward_hook(module, inp, out):
    if isinstance(out, tuple):
      out = out[0]
    out = out.view(out.size(0), -1)   
    x = (out > 0).float()  
    network.boolRelu = torch.cat((network.boolRelu, x), -1)
    
  network.boolRelu = torch.tensor([])
  network.boolRelu = network.boolRelu.to(device)

  for name, module in network.named_modules(): 
    if (isinstance(module, torch.nn.modules.activation.ReLU)):
      module.register_forward_hook(boolRelu_forward_hook)
              

  network(x)
  k = (network.boolRelu @ network.boolRelu.t()) + ((1. - network.boolRelu) @ (1. - network.boolRelu.t()))
  logdet = torch.linalg.slogdet(k)[1].cpu().detach()
  torch.cuda.empty_cache()
  return logdet


In [17]:
def NNDegree(searchspace, uid):

  strOp = searchspace.get_str(uid)
  listOp = strOp.split('|')
  nConv = strOp.count('conv')
  skip1 = [3,7]
  skip2 = [6]
  nS1 = ''.join([i for j, i in enumerate(listOp) if j in skip1]).count('skip')#skip 1 layer
  nS2  = ''.join([i for j, i in enumerate(listOp) if j in skip2]).count('skip')#skip 2 layers
  Sc = (nS1+2*nS2) #total skipped layers
  Wc = nConv
  return int((Wc + Sc))

In [18]:
def get_uid_and_measure(uid, input, metric_name, searchspace, population = None):
  scaler = StandardScaler()
  uid = int(uid)
  network = searchspace.get_network(uid)
  network.to(device)
  if metric_name == "Naswot":
    return uid, get_naswot(network, input).item()
  elif metric_name == "log_synflow":
    deg =  NNDegree(searchspace, uid)
    return uid, predictive.get_log_syn(network, input), deg
  else:
    return 0

In [19]:
from sklearn.preprocessing import StandardScaler

def sum_syn_naswot_degree(list_measures, input, searchspace, task, model_builder, evaluator):
  nas = []
  slope = []
  scaler = StandardScaler()
  list_measures[list_measures == -inf] = 0
  syn = list_measures[:,1].reshape(-1,1)
  syn_nor = scaler.fit_transform(list_measures[:,1].reshape(-1,1))
  degree = list_measures[:,2].reshape(-1,1)
  deg = scaler.fit_transform(list_measures[:,2].reshape(-1,1))
  for uid in list_measures[:,0]:
      network = searchspace.get_network(int(uid))
      network.to(device)
      nas.append([ get_naswot(network, input).item()])
      
  naswot = np.array(nas).reshape(-1,1)
  WNor = scaler.fit_transform(np.array(nas).reshape(-1,1))
  measures = np.hstack((list_measures[:,0].reshape(-1,1), syn_nor+WNor+deg, syn, naswot, degree)) 
  return measures

In [20]:
def get_uid_and_measures(uid_list, input, searchspace, metric_name):
  naswot = []
  log_Syn = []
  norm = []

  if metric_name == "Naswot":
    for uid in uid_list:
      network = searchspace.get_network(int(uid))
      network.to(device)
      naswot.append([uid, get_naswot(network, input).item()])
    return np.array(naswot)
  elif metric_name == "log_synflow":
    for uid in uid_list:
      deg =  NNDegree(searchspace, uid)
      network = searchspace.get_network(int(uid))
      log_Syn.append([uid, predictive.get_log_syn(network, input), deg])
    return np.array(log_Syn)
  else:
    return 0

In [21]:
def get_archs(searchspace,list_uid):
  list_arch = []
  for uid in list_uid:
    list_arch.append(searchspace.get_unique_str(uid))
  return list_arch

In [22]:
def lossSlope(losslist):
  trend = np.polyfit(range(len(losslist[1:10])),losslist[1:10],1)
  return trend[0]

In [27]:
def get_regression_score(uid, task, model_builder, evaluator, searchspace):
  arch = searchspace.get_str(uid)
  losses = evaluator.evaluate(task, model_builder, arch)
  return -lossSlope(list(losses))

In [28]:

from collections import defaultdict


def dominates(obj1, obj2, sign=[1, 1]):
    indicator = False
    for a, b, sign in zip(obj1, obj2, sign):
        if a * sign > b * sign:
            indicator = True
        # if one of the objectives is dominated, then return False
        elif a * sign < b * sign:
            return False
    return indicator


def sortNondominated(fitness, k=None, first_front_only=False):
   
    fitness = np.ndarray.tolist(fitness)
    if k is None:
        k = len(fitness)

    # Use objectives as keys to make python dictionary
    map_fit_ind = defaultdict(list)
    for i, f_value in enumerate(fitness):  # fitness = [(1, 2), (2, 2), (3, 1), (1, 4), (1, 1)...]
        map_fit_ind[tuple(f_value)].append(i)
    fits = list(map_fit_ind.keys())  # fitness values

    current_front = []
    next_front = []
    dominating_fits = defaultdict(int)  # n (The number of people dominate you)
    dominated_fits = defaultdict(list)  # Sp (The people you dominate)

    # Rank first Pareto front
    # *fits* is a iterable list of chromosomes. Each has multiple objectives.
    for i, fit_i in enumerate(fits):
        for fit_j in fits[i + 1:]:
            # Eventhougn equals or empty list, n & Sp won't be affected
            if dominates(fit_i, fit_j):
                dominating_fits[fit_j] += 1  
                dominated_fits[fit_i].append(fit_j)  
            elif dominates(fit_j, fit_i):  
                dominating_fits[fit_i] += 1
                dominated_fits[fit_j].append(fit_i)
        if dominating_fits[fit_i] == 0: 
            current_front.append(fit_i)

    fronts = [[]]  # The first front
    for fit in current_front:
        fronts[-1].extend(map_fit_ind[fit])
    pareto_sorted = len(fronts[-1])

    # Rank the next front until all individuals are sorted or
    # the given number of individual are sorted.
    # If Sn=0 then the set of objectives belongs to the next front
    if not first_front_only:  # first front only
        N = min(len(fitness), k)
        while pareto_sorted < N:
            fronts.append([])
            for fit_p in current_front:
                # Iterate Sn in current fronts
                for fit_d in dominated_fits[fit_p]: 
                    dominating_fits[fit_d] -= 1  # Next front -> Sn - 1
                    if dominating_fits[fit_d] == 0:  # Sn=0 -> next front
                        next_front.append(fit_d)
                         # Count and append chromosomes with same objectives
                        pareto_sorted += len(map_fit_ind[fit_d]) 
                        fronts[-1].extend(map_fit_ind[fit_d])
            current_front = next_front
            next_front = []

    return np.array(fronts)

In [30]:
def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    return np.argmin(dist_2)

In [31]:
class args:
  pass

args.total_iters = 10 #100
args.eval_interval = 1 #10
args.init_w_type = 'none'
args.init_b_type = 'none'
args.learning_rate = 1e-1
args.weight_decay = 4e-5
args.momentum = 0.9 
args.eval_weights = [0.25,0.5,1.]
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.train_weights = [0.25,0.5,1.]
args.config = "CONF_NB101"
args.workers = 2
args.pad = 'store_true'
args.input_size = 32 #cifar 32?
args.output_size = 8 #cifar 8?
args.last_channels = 64
args.batch_size = 16 #16
#CIFAR10
if Dataset_name == "Cifar-10":
  args.data_loc = 'insert path cifar-10'
  args.dataset = 'cifar10'
  args.num_labels = 10

#CIFAR100
if Dataset_name == "Cifar-100":
  args.dataset = 'cifar100'
  args.data_loc = "insert path cifar-100"
  args.num_labels = 100

#IMAGENET
if Dataset_name == "ImageNet":
  args.dataset = 'ImageNet16-120'
  args.data_loc = "insert path ImageNet16"
  args.num_labels = 120

args.init_channels = 16
args.search_space = 'nasbench201'

In [34]:
args.config = eval(args.config)

In [35]:
evaluator = Evaluator(args)
task = CVTask(args)
model_builder = ModelBuilder(args)

In [36]:
df_config = pd.read_csv("conf.csv") #dataframe of networks configuration

In [37]:
searchspace = nasspace.get_search_space(dataset, df_config.conf)
train_loader = datasets.get_data(dataset, data_loc, batch_size)

data_iterator = iter(train_loader)
input, target = next(data_iterator)
input, target = input.to(device), target.to(device)

[2022-08-28 19:17:02] Try to use the default NATS-Bench (topology) path from fast_mode=True and path=None.


# ET-NAS

In [ ]:
from numpy.random import default_rng

Number_experiments = 30
filename = f'ET-NAS_on_{dataset}_in_{Number_experiments}_experimets.csv'
arch_dict = []

arch_info = ['arch_id' , 'time', 'acc', 'test_acc']
history = []
from numpy.random import default_rng
population_size=20 
n_sample=5
cycles= 60

for j in range(0,Number_experiments):
  
  new_gen = []
  history = []


  start = timer()

  uid_population =  random.sample(range(len(searchspace)),population_size)

  population =  get_uid_and_measures(uid_population, input, searchspace, "log_synflow") 

  history = get_archs(searchspace, uid_population)

          
  while len(history) <= cycles:

      uid_sample = random.sample(range(len(population)),n_sample)

      sample = population[uid_sample]

      parent = max(sample[sample[:,2] == max(sample[:,2])], key=lambda i: i[1]) 

      child_idx = searchspace.mutate_arch(int(parent[0])) 

      unique_str_child = searchspace.get_unique_str(child_idx)

      degree_child = NNDegree(searchspace, child_idx)

      degree_parent = parent[-1]

      if unique_str_child not in history and degree_child >= degree_parent:


        uid_and_measure = np.array(get_uid_and_measure(child_idx, input, "log_synflow", searchspace, population))

        new_gen.append([int(uid_and_measure[0]), uid_and_measure[1], uid_and_measure[2] ])

        population = np.delete(population, 0, axis=0)

        population = np.vstack((population, uid_and_measure))

        history.append(unique_str_child)


     
          
          
  

  gen = np.array(new_gen)

  gen = gen[gen[:,2] == max(gen[:,2])] #consider the architectures with higher NNdegree

  measure= sum_syn_naswot_degree(gen, input, searchspace,task,model_builder,evaluator) 

  syn_naswot = measure[:,[2,3]]

  pareto_index = sortNondominated(syn_naswot, first_front_only=True)
  syn_naswot_pareto = syn_naswot[pareto_index[0]]
  arch_pareto = measure[pareto_index[0]][:,0]


  regression = []


  if len(arch_pareto) > 1:
    for uid in arch_pareto:
      regression.append(get_regression_score(uid, task, model_builder, evaluator, searchspace))
    
    syn_naswot_regression = np.hstack((syn_naswot_pareto, np.array(regression).reshape(-1,1)))
    measures_normalized = (syn_naswot_regression) / (syn_naswot_regression.max(axis = 0))
    best_arch_index = closest_node(np.array([1,1,1]), measures_normalized)
    best_arch = arch_pareto[best_arch_index]
    total_time_cost = (timer()-start)

  else:
    best_arch = arch_pareto[0]
    total_time_cost = (timer()-start)


  arch_dict.append({'arch_id' : int(best_arch), 'time': total_time_cost, 'test_acc': searchspace.get_final_accuracy(int(best_arch)) })

  with open(filename, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = arch_info)
    writer.writeheader()
    writer.writerows(arch_dict)






In [ ]:
OurRea_on_ImageNet16_120 = pd.read_csv("ET-NAS_on_dataset_in_Number_experiments_experimets.csv")#insert path of csv output
print(OurRea_on_ImageNet16_120.mean())
print(OurRea_on_ImageNet16_120.std())